# soilgrids

This package provides a minimal wrapper for the ISRIC Soilgrids API, allowing 
users to query soil properties by latitude/longitude and to perform basic 
analyses on the returned data.

Functions:
*   `get_soilgrids()`: Provides a simple wrapper for the /properties/query
    API endpoint, parsing the geojson response into a pandas DataFrame.
        
Classes:
*   `SoilGrids()`: Provides methods for reading data from Soilgrids and basic 
    including utilities for aggregating and analysing the returned data.

Useful links:
*   Documentation for the API: <https://rest.isric.org/soilgrids/v2.0/docs>
*   ISRIC REST entry page, including fair use policy: <https://rest.isric.org>
*   ISRIC data and software policy: <https://www.isric.org/about/data-policy>
*   Soilgrids FAQ: <https://www.isric.org/explore/soilgrids/faq-soilgrids>

## Querying data using `SoilGrids`:
The following code reads in the mean values for clay, sand, silt and ocs 
(organic carbon stock) in the top 30cm of soil for a random set of 50 points 
within roughly 25km of 
[Herning, Denmark](https://en.wikipedia.org/wiki/Herning). Note that points can 
be queried at a maximum rate of 5/minute, so the following code takes about 10 
minutes to run:

In [2]:
# ~~~ Console messages disabled for a cleaner output ~~~ #
import logging
logging.getLogger('soilgrids').setLevel(logging.ERROR)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

from soilgrids import SoilGrids
import pandas as pd


sg = SoilGrids()

sg.get_points_sample(
    50,
    lat_min=56.225297, lat_max=55.958103,
    lon_min=8.662215, lon_max=9.354390,
    soil_property=['clay', 'sand', 'silt', 'ocs'],
    depth=['0-5cm', '5-15cm', '15-30cm', '0-30cm'],
    value='mean'
)

sg.data \
    .filter([
        'lat', 'lon', 'soil_property', 'mapped_units', 
        'target_units', 'depth', 'mean'
    ])

,lat,lon,soil_property,mapped_units,target_units,depth,mean
0,56.276607,9.069328,clay,g/kg,%,0-5cm,58
1,56.276607,9.069328,clay,g/kg,%,5-15cm,52
2,56.276607,9.069328,clay,g/kg,%,15-30cm,64
3,56.276607,9.069328,ocs,t/ha,kg/m²,0-30cm,66
4,56.276607,9.069328,sand,g/kg,%,0-5cm,844
...,...,...,...,...,...,...,...
5,56.276492,8.923697,sand,g/kg,%,5-15cm,836
6,56.276492,8.923697,sand,g/kg,%,15-30cm,810
7,56.276492,8.923697,silt,g/kg,%,0-5cm,111
8,56.276492,8.923697,silt,g/kg,%,5-15cm,111


## Get the property (clay, sand, silt) with the highest value for each point
The `SoilGrids` class provides a handy utility `main_properties()` for finding
the most abundant soil type (i.e. property) for each point. In this case, we
see that the Herning region is quite sandy:

In [3]:
sg.main_properties()

,lat,lon,soil_property
0,56.226264,9.310044,sand
1,56.240131,9.085027,clay
2,56.240232,8.923844,sand
3,56.257524,8.701260,sand
4,56.263617,8.884656,sand
5,56.267173,9.272709,sand
6,56.269523,9.024149,sand
7,56.270786,9.056345,sand
8,56.271717,8.752865,sand
9,56.271802,8.721928,sand


## Relationship between clay, sand, silt and organic carbon stock
The `ocs_correlation()` method fits and displays summary statistics for a linear 
model using sand, clay and silt as predictors and OCS as the response variable. 
In this case, we find that these properties are not particularly correlated 
with OCS.

In [4]:
print(sg.ocs_correlation(capture_output=True))


Call:
lm(formula = clay + sand + silt ~ ocs, data = input_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-184.510  -57.234    6.529   54.289  182.565 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 113.2903    45.7361   2.477   0.0168 *  
ocs          16.0033     0.8548  18.721   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 84.13 on 48 degrees of freedom
Multiple R-squared:  0.8795,	Adjusted R-squared:  0.877 
F-statistic: 350.5 on 1 and 48 DF,  p-value: < 2.2e-16




## Disclaimers

*   Use of this package is subject to [ISRIC data and software policy](https://www.isric.org/about/data-policy).

*   This package is licensed as [GPL-2](LICENSE).